## Cargar modulos

In [1]:
import os
import sys

import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

Crear sesión de spark

In [2]:
spark : SparkSession= SparkSession.builder.appName("Mi primera app").getOrCreate()

In [3]:
spark

## Datos

Decidí utilizar estos datos por diversos motivos:
* Los datos son de dominio público y en la plataforma de kaggle tiene una calficación de 10 en usabilidad.
* Tiene una buena cantidad de columnas y registros.
* Pero debido a las capacidades de mi computuradora solo analizaré los datos del 2021,
    los cuales cuenta con 6,311,871 filas y 61 columnas. Pero cuenta con columnas redundantes que se eliminaran, en total se elegieron 28 columnas.
* Los datos representan vuelos en Estados Unidos de América, por lo tanto tiene una variedad de diferentes tipos de datos: número, texto, hora, etc.

Los datos representan vuelos comerciales en Estados Unidos durante los años 2018 al 2021. Pero solo elegí utilizar los datos del 2021 por los motivos previamente mencionados.

### Columnas

Las columnas que seleccioné son:

1. **Airline**: nombre de la aerolinea.
2. **CRSDepTime**: Hora de salida programada. (hhmm)
3. **DepTime**: Hora de salida verdadera. (hhmm)
4. **DepDelay**: Diferencia en minutos entre CRSDepTime y DepTime, tiempos negativos significa salidas tempranas.
5. **CRSArrTime**: Hora de llegada programada. (hhmm)
6. **ArrTime**: Hora de llegada verdadera. (hhmm)
7. **ArrDelay**: Diferencia en minutos entre la llegada programada y la hora de llegada, tiempos negativos significa llegadas tempranas.
8. **ActualElapsedTime**: Tiempo de vuelo verdadero.
9. **CRSElapsedTime**: Tiempo de vuelo programado.
10. **Distance**: Distancia entre aeropuertos en millas.
11. **Year**: Año del vuelo.
12. **Month**: Mes del vuelo.
13. **DayofMonth**: Día del mes del vuelo.
14. **DayOfWeek**: Día de la semana del vuelo.
15. **Tail_Number**: Número de la cola, código para identificar la aeronave.
16. **Flight_Number_Operating_Airline**: Número del vuelo.
17. **Origin**: Aeropuerto de origen.
18. **OriginAirportID**: Código del aeropuerto de origen.
19. **OriginAirportSeqID**: Código del aeropuerto de origen.
20. **OriginCityName**: Nombre de la ciudad del aeropuerto de origen.
21. **OriginStateName**: Nombre de estado del aeropuerto de origen.
22. **Dest**: Aeropuerto de destino.
23. **DestAirportID**: Código del aeropuerto de destino.
24. **DestAirportSeqID**: Código del aeropuerto de destino.
25. **DestCityName**: Nombre de la ciudad del aeropuerto de destino.
26. **DestStateName**: Nombre de estado del aeropuerto de destino.
27. **Cancelled**: Si el vuelo fue cancelado, 1 = Sí, 0 = No.
28. **Diverted**: Si el vuelo fue desviado, 1 = Sí, 0 = No.

## Cargar datos

In [4]:
import os
import zipfile
import pathlib

direccion = pathlib.Path.cwd()

path_datos_zip = pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\data.zip')

#es posible agregar mas archivos y luego juntarlos en un solo dataframe
path_datos_csv = pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2021.csv')

if not os.path.isfile(path_datos_csv):
    try:
        with zipfile.ZipFile(path_datos_zip, mode="r") as archivo_zip:
            archivo_zip.extract('Combined_Flights_2021.csv', path=pathlib.Path.joinpath(direccion.parent.parent,'Datos'))
    except:
        print(".zip no existe")
    


In [5]:
df: DataFrame = spark.read.csv(path_datos_csv.absolute().as_posix(), header=True, inferSchema=True)

In [6]:
df.printSchema()

root
 |-- FlightDate: date (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: boolean (nullable = true)
 |-- Diverted: boolean (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- DepDelayMinutes: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- ArrTime: double (nullable = true)
 |-- ArrDelayMinutes: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- CRSElapsedTime: double (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Marketing_Airline_Network: string (nullable = true)
 |-- Operated_or_Branded_Code_Share_Partners: string (nullable = true)
 |

In [7]:
#comprobacion de carga correcta con PySpark
df.describe().show()

+-------+--------------------+-------+-------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+-----------------+-----------------+-------+------------------+------------------+------------------+------------------+-------------------------+---------------------------------------+------------------------+---------------------------+-------------------------------+-----------------+------------------------+---------------------------+-----------+-------------------------------+------------------+------------------+------------------+--------------+-----------+------------------+---------------+------------------+------------------+-----------------+------------------+------------+---------+------------------+-------------+------------------+------------------+--------------------+----------+-----------------+------------------+------------------+-----------------+------------------+------

In [8]:
print((df.count(), len(df.columns)))

(6311871, 61)


In [9]:
#limitar a columnas más relevantes
columnas = [
    'Airline', # Aerolinea
    'CRSDepTime', #Hora de salida programada
    'DepTime', #Hora de salida actual
    'DepDelay', #Diferencia en minutos entre CRSDepTime y DepTime, tiempos negativos significa salidas tempranas
    'CRSArrTime',
    'ArrTime', #Hora de llegada
    'ArrDelay', #Diferencia de minutos entre la llegada programada y la hora de llegada, tiempos negativos significa llegadas tempranas
    'ActualElapsedTime', #Tiempo de vuelo verdadero
    'CRSElapsedTime', #Tiempo de vuelo programado
    'Distance', #Distancia entre aeropuertos en millas
    'Year', #Año del vuelo
    'Month', #Mes del vuelo
    'DayofMonth', #Dia del mes del vuelo
    'DayOfWeek', #Dia de la semana del vuelo
    'Tail_Number', #Numero de la cola, código para identificar la aeronave
    'Flight_Number_Operating_Airline', #Numero del vuelo
    
    'Origin', #Aeropuerto de origen
    'OriginAirportID', 'OriginAirportSeqID', #codigos del aeropuerto de origen
    'OriginCityName', #Nombre de la ciudad del aeropuerto de origen
    'OriginStateName', #Nombre de estado del aeropuerto de origen
    
    'Dest', #Aeropuerto de destino
    'DestAirportID', 'DestAirportSeqID', #codigos del aeropuerto de destino
    'DestCityName', #Nombre de la ciudad del aeropuerto de destino
    'DestStateName', #Nombre de estado del aeropuerto de destino
    
    'Cancelled', #El vuelo fue cancelado, 1 = Sí
    
    'Diverted', #Si el vuelo fue desviado, 1 = Sí
]

In [10]:
df: DataFrame = df.select(columnas)

In [11]:
print((df.count(), len(df.columns)))

(6311871, 28)


## Estadísticas descriptivas

Para calcular las estadísticas descriptivas, utilicé la función describe de pyspark.

In [16]:
df.describe().show()

+-------+--------------------+------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+-------+------------------+------------------+------------------+-----------+-------------------------------+-------+------------------+------------------+--------------+---------------+-------+------------------+-----------------+------------+-------------+
|summary|             Airline|        CRSDepTime|           DepTime|         DepDelay|        CRSArrTime|           ArrTime|          ArrDelay|ActualElapsedTime|   CRSElapsedTime|         Distance|   Year|             Month|        DayofMonth|         DayOfWeek|Tail_Number|Flight_Number_Operating_Airline| Origin|   OriginAirportID|OriginAirportSeqID|OriginCityName|OriginStateName|   Dest|     DestAirportID| DestAirportSeqID|DestCityName|DestStateName|
+-------+--------------------+------------------+------------------+-----------------+--

In [19]:
df.groupBy('Airline').avg('DepDelay').orderBy(F.desc("avg(DepDelay)")).show(truncate=False)

+-----------------------------------------+------------------+
|Airline                                  |avg(DepDelay)     |
+-----------------------------------------+------------------+
|Commutair Aka Champlain Enterprises, Inc.|17.817486088467913|
|JetBlue Airways                          |16.584311504495744|
|Allegiant Air                            |15.222554311460053|
|Mesa Airlines Inc.                       |14.7425790440584  |
|Southwest Airlines Co.                   |13.334239571459165|
|Frontier Airlines Inc.                   |11.490848204385173|
|Spirit Air Lines                         |11.451355395745415|
|American Airlines Inc.                   |11.408764971213344|
|Air Wisconsin Airlines Corp              |11.335232156832557|
|GoJet Airlines, LLC d/b/a United Express |11.131843710160906|
|SkyWest Airlines Inc.                    |10.056801801619242|
|United Air Lines Inc.                    |8.995028035348128 |
|Envoy Air                                |6.3709594294

Al calcular la media de de los minutos de retraso por aerolinea, se puede que existe un efecto por aerolinea.

In [30]:
df.groupBy('Diverted').avg('DepDelay').orderBy(F.desc("avg(DepDelay)")).show(truncate=False)

+--------+-----------------+
|Diverted|avg(DepDelay)    |
+--------+-----------------+
|true    |27.5812308103057 |
|false   |9.424496758167924|
+--------+-----------------+



Como es de esperar, los vuelos que fueron desviados en promedio tienen un retraso mayor a los que no lo fueron.

In [40]:
df.filter(df.Cancelled == 1).groupby('OriginStateName','OriginCityName').agg(F.count('OriginCityName').alias("Cantidad")).agg({'Cantidad':'avg'}).show(truncate=False)

+------------------+
|avg(Cantidad)     |
+------------------+
|301.67934782608694|
+------------------+



Cantidad promedio de vuelos cancelados por aeropuerto.

In [42]:
df.filter(df.Cancelled == 1).groupBy('OriginStateName','OriginCityName').count().orderBy(F.desc("count")).show(truncate=False)

+---------------+---------------------+-----+
|OriginStateName|OriginCityName       |count|
+---------------+---------------------+-----+
|Texas          |Dallas/Fort Worth, TX|8980 |
|Illinois       |Chicago, IL          |7508 |
|Colorado       |Denver, CO           |5839 |
|Texas          |Houston, TX          |4921 |
|New Jersey     |Newark, NJ           |3234 |
|North Carolina |Charlotte, NC        |2683 |
|Virginia       |Washington, DC       |2606 |
|New York       |New York, NY         |2582 |
|Washington     |Seattle, WA          |2399 |
|Florida        |Orlando, FL          |2197 |
|California     |Los Angeles, CA      |2177 |
|Nevada         |Las Vegas, NV        |2077 |
|Georgia        |Atlanta, GA          |2049 |
|Arizona        |Phoenix, AZ          |2024 |
|Louisiana      |New Orleans, LA      |1974 |
|Maryland       |Baltimore, MD        |1892 |
|Florida        |Fort Lauderdale, FL  |1642 |
|Texas          |Dallas, TX           |1581 |
|Tennessee      |Nashville, TN    

In [43]:
8980 / 301.67934782608694

29.766704498369634

Al contar los vuelos cancelados por aeropuerto, podemos ver que el aeropuerto con más vuelos cancelados, está por encima del promedio por 29.7 veces.

In [15]:
df.groupBy('Tail_Number').avg('DepDelay').orderBy(F.desc("avg(DepDelay)")).show(truncate=False)

+-----------+------------------+
|Tail_Number|avg(DepDelay)     |
+-----------+------------------+
|N567NK     |655.0             |
|N360NK     |223.0             |
|N506DN     |53.77777777777778 |
|N512MJ     |49.246153846153845|
|N838UA     |48.04347826086956 |
|N507DN     |44.857142857142854|
|N59053     |44.641975308641975|
|N676UA     |42.774193548387096|
|N761AJ     |41.58893280632411 |
|N756AM     |39.986970684039086|
|N78005     |38.94117647058823 |
|N717AN     |38.40329218106996 |
|n714FR     |38.0              |
|N228PS     |37.97959183673469 |
|N29968     |37.279411764705884|
|N813NW     |37.056            |
|N733AR     |36.477366255144034|
|N954SW     |36.090425531914896|
|N948NK     |35.59090909090909 |
|N661UA     |34.92857142857143 |
+-----------+------------------+
only showing top 20 rows



Como se puede observar existen 2 aeronaves las cuales tienen un tiempo promedio de retraso muy elevado.

Tarea 1 (10 puntos). Creación y operaciones básicas con PySpark

* Instalar en un entorno local o ejecutar Spark en algún servidor en línea (como Google Colab)
* Elegir un conjunto de datos para trabajar durante el tetramestre, definirlo y explicar por qué se elige
* Cargar el conjunto de datos mediante PySpark
* Usar PySpark para filtrar datos, generar estadísticas descriptivas básicas y realizar algunas operaciones aritméticas entre registros y columnas
* Crear un repositorio público para el curso y publicar en un cuaderno esta primera tarea